# Cleaning Data

In [63]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [64]:
# loading the data
df_1 = pd.read_csv("C:/Users/Rohit Gupta/Downloads/CAvideos.csv")
df_2 = pd.read_csv("C:/Users/Rohit Gupta/Downloads/AB_NYC_2019.csv")

In [65]:
# Check for data types and missing values
print(df_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40881 entries, 0 to 40880
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40881 non-null  object
 1   trending_date           40881 non-null  object
 2   title                   40881 non-null  object
 3   channel_title           40881 non-null  object
 4   category_id             40881 non-null  int64 
 5   publish_time            40881 non-null  object
 6   tags                    40881 non-null  object
 7   views                   40881 non-null  int64 
 8   likes                   40881 non-null  int64 
 9   dislikes                40881 non-null  int64 
 10  comment_count           40881 non-null  int64 
 11  thumbnail_link          40881 non-null  object
 12  comments_disabled       40881 non-null  bool  
 13  ratings_disabled        40881 non-null  bool  
 14  video_error_or_removed  40881 non-null  bool  
 15  de

In [66]:
print(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [67]:
# checking for missing values
print(df_1.isnull().sum())

video_id                     0
trending_date                0
title                        0
channel_title                0
category_id                  0
publish_time                 0
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
thumbnail_link               0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
description               1296
dtype: int64


In [68]:
print(df_2.isnull().sum())

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


### The second dataset contains missing values. So, i am going to fill these missing values using mean.

In [69]:
# Selecting columns with missing values
cols_with_missing = ['name', 'host_name', 'last_review', 'reviews_per_month']

# Impute missing values with the most frequent value for object columns and mean for float column
imputer = SimpleImputer(strategy='most_frequent')  # for object columns
df_2[cols_with_missing[:3]] = imputer.fit_transform(df_2[cols_with_missing[:3]])

imputer = SimpleImputer(strategy='mean')  # for float column
df_2[cols_with_missing[3]] = imputer.fit_transform(df_2[cols_with_missing[3]].values.reshape(-1, 1))

print(df_2.isnull().sum())

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64


### Now both the dataset doesn't contain any missing values.

## Checking Duplicate Values

In [70]:
print(df_1[df_1.duplicated()])

print(df_2[df_2.duplicated()])

Empty DataFrame
Columns: [video_id, trending_date, title, channel_title, category_id, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, video_error_or_removed, description]
Index: []
Empty DataFrame
Columns: [id, name, host_id, host_name, neighbourhood_group, neighbourhood, latitude, longitude, room_type, price, minimum_nights, number_of_reviews, last_review, reviews_per_month, calculated_host_listings_count, availability_365]
Index: []


### No duplicates are present in both datasets.

In [71]:
# Standardizing numerical features
numerical_columns = df_1.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df_1[numerical_columns] = scaler.fit_transform(df_1[numerical_columns])

In [72]:
numerical_columns = df_2.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df_2[numerical_columns] = scaler.fit_transform(df_2[numerical_columns])

# Outlier Detection

In [73]:
def detect_outliers(df, cols, contamination=0.1):
    """
    Detect outliers using Isolation Forest
    """
    clf = IsolationForest(contamination=contamination, random_state=42)
    outliers = []
    for col in cols:
        clf.fit(df[col].values.reshape(-1, 1))
        outlier_indices = df[col][clf.predict(df[col].values.reshape(-1, 1)) == -1].index
        outliers.extend(outlier_indices)
    return list(set(outliers))

In [74]:
numerical_cols = df_1.select_dtypes(include=['float', 'int']).columns
outliers_df_1 = detect_outliers(df_1, numerical_cols, contamination=0.1)

print("\nOutliers in Dataset 1 (Isolation Forest):")
print(outliers_df_1)


Outliers in Dataset 1 (Isolation Forest):
[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 32780, 32783, 18, 19, 22, 23, 32790, 32791, 32794, 32796, 32797, 32, 32800, 32804, 32805, 40, 32808, 32810, 43, 32812, 45, 44, 32815, 48, 49, 32820, 53, 55, 32823, 32833, 66, 32840, 32841, 74, 32849, 32853, 86, 87, 90, 32859, 32869, 103, 32871, 111, 113, 114, 32883, 32887, 121, 122, 32892, 124, 32894, 127, 32897, 32899, 134, 32904, 32914, 32915, 32916, 32917, 32920, 153, 156, 32925, 32924, 32927, 159, 32931, 164, 165, 32934, 32933, 168, 32936, 170, 32939, 32937, 167, 32942, 32943, 174, 32946, 32949, 32951, 32952, 32953, 187, 32958, 32959, 195, 32965, 32967, 200, 201, 203, 204, 206, 209, 211, 212, 32980, 214, 32983, 218, 32987, 219, 32991, 32992, 226, 227, 228, 32994, 32996, 231, 233, 33001, 33004, 236, 240, 33011, 244, 33017, 251, 252, 253, 33022, 33025, 258, 259, 33026, 33027, 262, 264, 33033, 33034, 273, 33046, 33049, 282, 33051, 33053, 291, 33061, 295, 33070, 304, 305, 33074, 307, 33076, 33072, 33

In [75]:
numerical_cols = df_2.select_dtypes(include=['float', 'int']).columns
outliers_df_2 = detect_outliers(df_2, numerical_cols, contamination=0.1)
print("\nOutliers in Dataset 2 (Isolation Forest):")
print(outliers_df_2)


Outliers in Dataset 2 (Isolation Forest):
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 2

### Outliers are present in both datasets. So, i am going to remove these outliers as presence of outliers can skew the analysis.

In [76]:
# Removing outliers from datasets
df_1 = df_1.drop(outliers_df_1)
df_2 = df_2.drop(outliers_df_2)

In [77]:
print(df_1.head())

       video_id trending_date  \
9   43sm-QwLcx4      17.14.11   
14  LUzsOyWp9lw      17.14.11   
15  AS9-ITLhQxo      17.14.11   
16  gifPYwArCVQ      17.14.11   
17  8NHA23f7LvU      17.14.11   

                                                title           channel_title  \
9   Finally Sheldon is winning an argument about t...             Sheikh Musa   
14  YOUTUBERS REACT TO TOP 10 TWITTER ACCOUNTS OF ...                     FBE   
15            I Hired An MI6 Spy To Help Me Disappear            BuzzFeedBlue   
16                     Fake Pet Smart Employee Prank!                    NELK   
17  Jason Momoa Wows Hugh Grant With Some Dothraki...  The Graham Norton Show   

    category_id              publish_time  \
9      0.177779  2017-11-10T14:10:46.000Z   
14     0.472983  2017-11-12T22:00:01.000Z   
15     0.177779  2017-11-11T16:00:44.000Z   
16     0.325381  2017-11-13T01:30:01.000Z   
17     0.472983  2017-11-10T19:06:23.000Z   

                                           

In [78]:
print(df_2.head())

            id                                               name   host_id  \
1498 -1.669977                  AMAZING LOCATION YOU WILL LOVE IT -0.816526   
1499 -1.669935                  small private bedroom female only -0.841551   
1503 -1.669584                               Newly Renovated 1 BR -0.816151   
1504 -1.669573  Large private bedroom in house, Bushwick/Ridge... -0.826153   
1513 -1.669246   Spacious Private room in beautiful 1BR near Park -0.816341   

     host_name neighbourhood_group    neighbourhood  latitude  longitude  \
1498    Callie           Manhattan     West Village  0.102901  -1.111450   
1499      Ouii           Manhattan         Kips Bay  0.233105  -0.562012   
1503   Stephen           Manhattan  Lower East Side -0.173280  -0.909744   
1504     Alice              Queens        Ridgewood -0.372072   0.797283   
1513       Mie           Manhattan           Harlem  1.360568  -0.119385   

         room_type     price  minimum_nights  number_of_reviews last